### Lasso Regression

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import plotly.express as px
from sklearn.model_selection import cross_val_score

In [38]:
# df = pd.read_csv("/Users/asheshlalshrestha/Desktop/UB/Research/CarePathways/Models/Datasets/preprocessed_dataset.csv")
df = pd.read_csv("Datasets/new_dataset_added_column_scaled.csv")
df = df.drop(["Unnamed: 0","id"],axis=1)
df

,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,Chronic cardiac disease (not hypertension),...,pao2,pao2_fio2,ph,esr,inr,ferritin,d_dimer,crp,admission_disposition_encoded,intubated_encoded
0,21,0.486777,1,3.549903e-15,0.000000,0.0,0,1,1,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,-1.116803e-01,0.0,0.000000,2.887190e+00,1,0
1,5,-0.306970,0,-1.500191e+00,-0.364575,0.0,0,1,1,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,-2.998049e-01,0.0,1.595746,-3.430231e-01,1,0
2,7,-0.490142,0,3.549903e-15,0.000000,0.0,0,1,1,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.0,0.000000,-3.603147e-16,1,0
3,9,1.707926,1,1.984550e+00,-0.769298,0.0,0,1,1,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.0,0.000000,-3.603147e-16,1,0
4,9,1.524754,1,3.549903e-15,0.000000,1.0,0,1,1,1,...,0.000000,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.0,0.000000,-3.603147e-16,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,19,0.914179,1,-1.112997e+00,-0.834928,0.0,1,1,1,0,...,-0.468484,0.0,2.861595e+00,2.340288e-15,4.177205e-16,0.0,4.202592,-3.603147e-16,1,0
504,9,0.486777,1,3.549903e-15,0.078433,0.0,1,1,1,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,6.408181e-01,0.0,0.000000,-3.603147e-16,1,0
505,7,-1.100717,0,1.634827e+00,0.822248,0.0,1,1,0,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,-2.998049e-01,0.0,-1.454530,-3.603147e-16,1,0
506,5,0.425720,1,3.549903e-15,0.000000,0.0,1,1,0,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,-1.116803e-01,0.0,0.376269,-3.603147e-16,1,0


### Splitting X and y

In [39]:
from sklearn.model_selection import train_test_split
X = df.drop("hospital_length_of_stay",axis =1)
y = df['hospital_length_of_stay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (355, 782)
X_test shape: (153, 782)
y_train shape: (355,)
y_test shape: (153,)


In [40]:
X

,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,Chronic cardiac disease (not hypertension),Diabetes,...,pao2,pao2_fio2,ph,esr,inr,ferritin,d_dimer,crp,admission_disposition_encoded,intubated_encoded
0,0.486777,1,3.549903e-15,0.000000,0.0,0,1,1,0,1,...,0.000000,0.0,8.243049e-14,2.340288e-15,-1.116803e-01,0.0,0.000000,2.887190e+00,1,0
1,-0.306970,0,-1.500191e+00,-0.364575,0.0,0,1,1,0,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,-2.998049e-01,0.0,1.595746,-3.430231e-01,1,0
2,-0.490142,0,3.549903e-15,0.000000,0.0,0,1,1,0,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.0,0.000000,-3.603147e-16,1,0
3,1.707926,1,1.984550e+00,-0.769298,0.0,0,1,1,0,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.0,0.000000,-3.603147e-16,1,0
4,1.524754,1,3.549903e-15,0.000000,1.0,0,1,1,1,1,...,0.000000,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.0,0.000000,-3.603147e-16,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,0.914179,1,-1.112997e+00,-0.834928,0.0,1,1,1,0,1,...,-0.468484,0.0,2.861595e+00,2.340288e-15,4.177205e-16,0.0,4.202592,-3.603147e-16,1,0
504,0.486777,1,3.549903e-15,0.078433,0.0,1,1,1,0,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,6.408181e-01,0.0,0.000000,-3.603147e-16,1,0
505,-1.100717,0,1.634827e+00,0.822248,0.0,1,1,0,0,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,-2.998049e-01,0.0,-1.454530,-3.603147e-16,1,0
506,0.425720,1,3.549903e-15,0.000000,0.0,1,1,0,0,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,-1.116803e-01,0.0,0.376269,-3.603147e-16,1,0


In [41]:
X_train

,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,Chronic cardiac disease (not hypertension),Diabetes,...,pao2,pao2_fio2,ph,esr,inr,ferritin,d_dimer,crp,admission_disposition_encoded,intubated_encoded
317,1.402639,1,3.549903e-15,0.000000,0.0,1,1,1,0,0,...,0.0,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.000000,0.000000,-3.603147e-16,1,0
479,-0.306970,0,-8.631952e-01,-0.966190,0.0,0,1,0,0,0,...,0.0,0.0,8.243049e-14,2.340288e-15,-2.998049e-01,0.000000,0.000000,5.798949e-01,1,0
323,-0.490142,0,3.549903e-15,-0.419267,0.0,0,1,0,0,1,...,0.0,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.000000,0.000000,-3.603147e-16,1,0
5,-1.344947,1,1.260124e+00,1.735610,0.0,0,0,1,0,0,...,0.0,0.0,8.243049e-14,2.340288e-15,-1.116803e-01,-2.431417,-0.032335,3.166094e+00,0,1
116,1.402639,1,3.549903e-15,0.000000,0.0,0,1,1,1,0,...,0.0,0.0,8.243049e-14,2.340288e-15,8.289427e-01,0.000000,-0.363336,-3.603147e-16,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1.646869,1,3.549903e-15,0.000000,0.0,0,1,0,0,0,...,0.0,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.000000,0.000000,-3.603147e-16,1,0
270,0.120432,1,3.549903e-15,0.000000,0.0,0,1,1,0,0,...,0.0,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.000000,-1.392764,-1.980949e+00,1,0
348,1.097352,1,1.997040e+00,0.570664,0.0,0,1,1,0,0,...,0.0,0.0,8.243049e-14,2.340288e-15,-2.998049e-01,0.000000,-0.268312,-3.603147e-16,1,0
435,0.547834,1,8.605319e-02,-0.840398,1.0,0,1,1,0,0,...,0.0,0.0,8.243049e-14,2.340288e-15,-4.879295e-01,0.000000,1.633756,-3.603147e-16,1,0


### Implenting lasso regression to the original dataset

In [42]:
from sklearn.linear_model import Lasso

# Apply Lasso Regression
lasso = Lasso(alpha=0.1)  # Specify the regularization parameter alpha
lasso.fit(X_train, y_train)
df_importance = pd.DataFrame({'Feature': X.columns, 'Importance': lasso.coef_}).sort_values(by='Importance', ascending=False)
important_features = df_importance[df_importance['Importance']>0]
important_features.reset_index()

,index,Feature,Importance
0,554,Isordil,15.900822
1,496,Lansoprazole,15.042524
2,123,Polyethylene Glycol,7.883947
3,697,Ferrous Gluconate Tab,7.714556
4,257,Cipralex,4.230607
5,0,age,2.369821
6,779,crp,2.174688
7,12,Kidney disease,1.967207
8,771,lactate,1.709541
9,767,serum_creatinine,1.572481


### Important features are those which have importance of more than 0

In [43]:
# Print feature importances
important_features = df_importance[df_importance['Importance']>0]
print("Total important features:",len(important_features))
important_features

Total important features: 34


,Feature,Importance
554,Isordil,15.900822
496,Lansoprazole,15.042524
123,Polyethylene Glycol,7.883947
697,Ferrous Gluconate Tab,7.714556
257,Cipralex,4.230607
0,age,2.369821
779,crp,2.174688
12,Kidney disease,1.967207
771,lactate,1.709541
767,serum_creatinine,1.572481


In [44]:
important_features.to_csv("lasso_important_features.csv")

### Dropping the unimportant features from X

In [45]:
feature_array = important_features['Feature'].values

cols_to_drop = [col for col in X.columns if col not in feature_array]

new_x_for_lasso = X.drop(cols_to_drop,axis=1)
new_x_for_lasso

,age,Hypertension,Kidney disease,reason_for_admission_COVID-19 [U07.1],reason_for_admission_Pneumonia [J18.9],reason_for_admission_Respiratory distress [R06.0],Amlodipine,Polyethylene Glycol,Eliquis,Janumet,...,ast,serum_creatinine,potassium,total_serum_bilirubin,lactate,pao2,ph,esr,ferritin,crp
0,0.486777,1,0,0,0,0,0.0,0.0,0.0,0.0,...,-0.177523,0.547415,0.027032,-0.124363,-2.919920e-02,0.000000,8.243049e-14,2.340288e-15,0.0,2.887190e+00
1,-0.306970,1,0,0,1,0,0.0,0.0,0.0,0.0,...,0.094499,-0.716990,-0.162943,0.270268,-7.910090e-01,0.000000,8.243049e-14,2.340288e-15,0.0,-3.430231e-01
2,-0.490142,1,0,0,1,0,0.0,0.0,0.0,0.0,...,0.209034,-0.477189,-0.352918,-0.716309,-1.267140e+00,0.000000,8.243049e-14,2.340288e-15,0.0,-3.603147e-16
3,1.707926,1,0,0,0,0,0.0,0.0,0.0,0.0,...,-0.535447,0.340314,-2.062691,-0.124363,2.065778e+00,0.000000,8.243049e-14,2.340288e-15,0.0,-3.603147e-16
4,1.524754,1,0,0,0,0,0.0,0.0,0.0,0.0,...,-0.506813,-0.062988,-0.162943,1.651478,8.278368e-01,0.000000,8.243049e-14,2.340288e-15,0.0,-3.603147e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,0.914179,1,0,1,0,0,0.0,0.0,0.0,0.0,...,0.123133,0.362114,0.596956,0.072953,1.303968e+00,-0.468484,2.861595e+00,2.340288e-15,0.0,-3.603147e-16
504,0.486777,1,0,1,0,0,0.0,0.0,0.0,0.0,...,-0.105938,-0.411789,-0.162943,-0.321678,-4.228894e-16,0.000000,8.243049e-14,2.340288e-15,0.0,-3.603147e-16
505,-1.100717,0,0,1,0,0,0.0,0.0,0.0,0.0,...,-0.406594,-0.400889,0.027032,0.072953,-8.862352e-01,0.000000,8.243049e-14,2.340288e-15,0.0,-3.603147e-16
506,0.425720,0,0,1,0,0,0.0,0.0,0.0,0.0,...,-0.349326,-0.389989,-0.162943,-0.716309,-4.228894e-16,0.000000,8.243049e-14,2.340288e-15,0.0,-3.603147e-16


### Using the important features in the model 

In [46]:
# Assuming new_x_for_lasso and y are already defined
X_train, X_test, y_train, y_test = train_test_split(new_x_for_lasso, y, test_size=0.3, random_state=42)

# Perform 5-fold cross-validation on the training data
cv_scores = cross_val_score(lasso, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Fit the model on the entire training data
lasso.fit(X_train, y_train)

# Predict on training and test data
y_pred_train = lasso.predict(X_train)
y_pred_test = lasso.predict(X_test)

# Calculate Mean Squared Error for training and test data
mse_y_train = mean_squared_error(y_train, y_pred_train)
mse_y_test = mean_squared_error(y_test, y_pred_test)

print("Cross-validation MSE scores:", -cv_scores)
print("Mean cross-validation MSE:", -np.mean(cv_scores))
print("Standard deviation of cross-validation MSE:", np.std(cv_scores))
print("Mean Squared Error train:", mse_y_train)
print("Mean Squared Error test:", mse_y_test)

Cross-validation MSE scores: [105.24088235 124.895203   181.92274605 133.11072826 128.01952162]
Mean cross-validation MSE: 134.63781625394756
Standard deviation of cross-validation MSE: 25.460151780913943
Mean Squared Error train: 96.53426638064535
Mean Squared Error test: 166.25743127665592


In [47]:
df_train = pd.DataFrame({'y_train': y_train, 'y_pred_train': y_pred_train})

# Create a scatter plot using Plotly Express
fig = px.scatter(df_train, x='y_train', y='y_pred_train', title='y_train vs y_pred_train')
fig.update_layout(xaxis_title='y_train', yaxis_title='y_pred_train')
fig.show()

df_test = pd.DataFrame({'y_test': y_test, 'y_pred_test': y_pred_test})

# Create a scatter plot using Plotly Express
fig = px.scatter(df_test, x='y_test', y='y_pred_test', color_discrete_sequence=['green'], title='y_test vs y_pred_test')
fig.update_layout(xaxis_title='y_test', yaxis_title='y_pred_test')
fig.show()

### Working on the final df to show the predicted and the actual values

In [48]:
## final_df is the df consisting of the id and the original data values
final_df = pd.read_csv('Datasets/new.csv')
final_df = final_df.drop(["Unnamed: 0"],axis=1)

In [49]:
final_df.head()

,id,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,...,Respiractin,Immunity Advance,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil
0,1,21,74,1,166.911029,80.265922,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,5,61,0,154.900000,73.600000,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,7,58,0,166.911029,80.265922,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,9,94,1,182.800000,66.200000,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,9,91,1,166.911029,80.265922,1.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
## we already have dataframe named df_test and df_train created above to plot the scatter plot

## we will rename the columns initially
df_test.rename(columns={'y_test': 'HLOS','y_pred_test':'predicted'}, inplace=True)
df_train.rename(columns={'y_train': 'HLOS','y_pred_train':'predicted'}, inplace=True)

## concat the two dataframes
combined_df = pd.concat([df_test, df_train])

## finally merge the combined_df based on index with the with the final_df
final_df = pd.merge(final_df, combined_df, left_index=True, right_index=True, suffixes=('', '_combined'))

## dropping the column hospital_length_of_stay to avoid confusion as HOLS has already been added 
final_df = final_df.drop('hospital_length_of_stay',axis = 1)
final_df.head()

,id,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,Chronic cardiac disease (not hypertension),...,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil,HLOS,predicted
0,1,74,1,166.911029,80.265922,0.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,17.018685
1,2,61,0,154.900000,73.600000,0.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,7.685571
2,3,58,0,166.911029,80.265922,0.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,7.074825
3,4,94,1,182.800000,66.200000,0.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,19.884728
4,5,91,1,166.911029,80.265922,1.0,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,14.915534


In [51]:
# final_df.to_csv("Datasets/df_with_predicted_values")

### Limiting the dataset

In [55]:
df_cap = df[df['hospital_length_of_stay']<20]
df_cap

,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,Chronic cardiac disease (not hypertension),...,pao2,pao2_fio2,ph,esr,inr,ferritin,d_dimer,crp,admission_disposition_encoded,intubated_encoded
1,5,-0.306970,0,-1.500191e+00,-0.364575,0.0,0,1,1,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,-2.998049e-01,0.0,1.595746,-3.430231e-01,1,0
2,7,-0.490142,0,3.549903e-15,0.000000,0.0,0,1,1,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.0,0.000000,-3.603147e-16,1,0
3,9,1.707926,1,1.984550e+00,-0.769298,0.0,0,1,1,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.0,0.000000,-3.603147e-16,1,0
4,9,1.524754,1,3.549903e-15,0.000000,1.0,0,1,1,1,...,0.000000,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.0,0.000000,-3.603147e-16,1,0
6,4,-0.978602,1,2.634036e+00,0.592541,0.0,0,1,0,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,4.177205e-16,0.0,-1.704776,-1.511884e+00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,19,0.914179,1,-1.112997e+00,-0.834928,0.0,1,1,1,0,...,-0.468484,0.0,2.861595e+00,2.340288e-15,4.177205e-16,0.0,4.202592,-3.603147e-16,1,0
504,9,0.486777,1,3.549903e-15,0.078433,0.0,1,1,1,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,6.408181e-01,0.0,0.000000,-3.603147e-16,1,0
505,7,-1.100717,0,1.634827e+00,0.822248,0.0,1,1,0,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,-2.998049e-01,0.0,-1.454530,-3.603147e-16,1,0
506,5,0.425720,1,3.549903e-15,0.000000,0.0,1,1,0,0,...,0.000000,0.0,8.243049e-14,2.340288e-15,-1.116803e-01,0.0,0.376269,-3.603147e-16,1,0


In [56]:
from sklearn.model_selection import train_test_split
X = df_cap.drop("hospital_length_of_stay",axis =1)
y = df_cap['hospital_length_of_stay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (292, 782)
X_test shape: (126, 782)
y_train shape: (292,)
y_test shape: (126,)


In [57]:
from sklearn.linear_model import Lasso

# Perform 5-fold cross-validation on the training data
cv_scores = cross_val_score(lasso, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Fit the model on the entire training data
lasso.fit(X_train, y_train)

# Predict on training and test data
y_pred_train = lasso.predict(X_train)
y_pred_test = lasso.predict(X_test)

# Calculate Mean Squared Error for training and test data
mse_y_train = mean_squared_error(y_train, y_pred_train)
mse_y_test = mean_squared_error(y_test, y_pred_test)

print("Cross-validation MSE scores:", -cv_scores)
print("Mean cross-validation MSE:", -np.mean(cv_scores))
print("Standard deviation of cross-validation MSE:", np.std(cv_scores))
print("Mean Squared Error train:", mse_y_train)
print("Mean Squared Error test:", mse_y_test)

Cross-validation MSE scores: [18.66104974 25.38148192 19.13046075 16.20324916 17.12537469]
Mean cross-validation MSE: 19.30032325158535
Standard deviation of cross-validation MSE: 3.21683956207595
Mean Squared Error train: 14.6019281345003
Mean Squared Error test: 22.244229326126572
